# Time Scheduler for Campus Classes

Time scheduler is a `Generative-AI` based app, developed using OpenAI `gpt-4` model. It takes subjects and students data in CSV format and generates time schedule for each individual student.

In [2]:
pip install openai python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import csv

def read_data_from_csv(filename):
    data = []

    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)

    return data

In [5]:
subjects_filename = "subjects.csv"
subjects_data = read_data_from_csv(subjects_filename)

In [6]:
students_filename = "students.csv"
students_data = read_data_from_csv(students_filename)

In [7]:
def perform_api_request(conversation, user_message, history=True):
    # Append the new user message to the conversation history
    conversation.append({"role": "user", "content": user_message})

    # Generate a chat completion response
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=conversation,
        n=1,
        temperature=0
    )

    # Extract the reply from the API response
    reply_message = response["choices"][0]["message"]["content"]

    if history:
        # Append the assistant's reply to the conversation history
        conversation.append({"role": "assistant", "content": reply_message})
    else:
        # remove user_message from the conversation
        conversation.pop()    

    # Return the assistant's reply
    return reply_message

In [8]:
conversation = [
    {"role": "system", "content": "You are an expert time scheduler for the university classes. You can also interpret JSON efficiently."}
]

In [9]:
initial_prompt = f"""
Here is the subjects data containing subject name, their total number of classes to be attended by a student, and the available classes slots.
{subjects_data}
Please remember that and answer upcoming questions
"""
response = perform_api_request(conversation, initial_prompt)

In [11]:
for student in students_data:
    prompt = f"""
    Here are the subjects registered and considerations for a student, please schedule classes for him for his given subjects.
    {student}
    The number of classes per week for each subject must be the same as mentioned in the data.
    Note that, if there is a conflict between scheduling and considerations, ignore the consideration. At the end of the message, add a very short and specific comment regarding consideration fulfillment.
    Moreover, never schedule two different classes at the same time.
    Prepare the timetable in the following format:
    StudentName:
    Subject1:
    Classes list
    Subject2:
    Classes list
    Subject3:
    Classes list
    and so on.
    """
    response = perform_api_request(
        conversation=conversation, 
        user_message=prompt, 
        history=False
    )
    print(response, '\n')

Based on the subjects data and the student's registered subjects, here is the proposed schedule for John Doe:

StudentName: John Doe

Subject1: Mathematics
Classes list:
- Monday 9:00 AM - 10:30 AM
- Tuesday 1:00 PM - 2:30 PM
- Thursday 1:00 PM - 2:30 PM

Subject2: Science
Classes list:
- Tuesday 9:00 AM - 10:30 AM
- Thursday 11:00 AM - 12:30 PM
- Friday 1:00 PM - 2:30 PM

Subject3: History
Classes list:
- Monday 11:00 AM - 12:30 PM
- Wednesday 1:00 PM - 2:30 PM

Note: The consideration of avoiding classes on Mondays and Fridays could not be fully met due to the limited available slots for the registered subjects. 

Based on the subjects registered and the considerations given, here is the proposed schedule for Jane Smith:

StudentName: Jane Smith

Subject1: English
Classes list:
- Tuesday 9:00 AM - 10:30 AM
- Tuesday 1:00 PM - 2:30 PM
- Wednesday 9:00 AM - 10:30 AM

Subject2: Mathematics
Classes list:
- Monday 9:00 AM - 10:30 AM
- Tuesday 3:00 PM - 4:30 PM
- Thursday 1:00 PM - 2:30 PM